<a href="https://colab.research.google.com/github/drAshcroft/LigandDiffusion/blob/main/ligandMPNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output
!pip install biopython
!pip install py3Dmol
!git clone https://github.com/dauparas/LigandMPNN.git
clear_output()

In [ ]:
%cd LigandMPNN
!bash get_model_params.sh "./model_params"

#setup your conda/or other environment
#conda create -n ligandmpnn_env python=3.11
!pip3 install -r requirements.txt
clear_output()

In [ ]:
import os
from IPython.display import clear_output
import random
from Bio.PDB import PDBParser, PDBIO, Selection
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import numpy as np
from scipy.optimize import minimize
import glob
from google.colab import drive
from Bio.PDB import Residue, Atom
import json
import random
drive.mount('/content/drive')
%cd LigandMPNN

inputFolder = '/content/LigandMPNN/inputs'


Mounted at /content/drive
/content/LigandMPNN


In [ ]:
# prompt: delete all the files in the inputFolder

import shutil
for filename in os.listdir(inputFolder):
    file_path = os.path.join(inputFolder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))


In [ ]:
import numpy as np
import random
import os
from Bio.PDB import *
import glob
from scipy.spatial.transform import Rotation

# Define the maximum displacement (in Angstroms) and rotation (in degrees)
max_displacement = 2.0
max_rotation = 180.0  # Full rotation possible

# Define the number of iterations
num_iterations = 30

# Assuming ./inputs is the directory containing the files
input_files = glob.glob("./inputs/working2_*.pdb")  # Adjust the pattern if needed

outputFiles = []
output_dir = "./inputs"

def get_heme_center_and_normal(residue):
    """Calculate the heme center (Fe) and normal vector from nitrogen atoms"""
    fe_atom = None
    n_atoms = []

    for atom in residue:
        if atom.name == "FE":
            fe_atom = atom
        elif atom.name in ["NA", "NB", "NC", "ND"]:
            n_atoms.append(atom)

    if not fe_atom or len(n_atoms) != 4:
        raise ValueError("Couldn't find Fe atom or all four nitrogen atoms")

    # Calculate the normal vector using three nitrogen atoms
    v1 = n_atoms[1].coord - n_atoms[0].coord
    v2 = n_atoms[2].coord - n_atoms[0].coord
    normal = np.cross(v1, v2)
    normal = normal / np.linalg.norm(normal)

    return fe_atom.coord, normal

def rotate_around_axis(coords, center, axis, angle_degrees):
    """Rotate coordinates around an axis passing through center"""
    # Convert angle to radians
    angle = np.radians(angle_degrees)

    # Create rotation matrix using scipy
    rotation_vector = axis * angle
    rotation = Rotation.from_rotvec(rotation_vector)
    rotation_matrix = rotation.as_matrix()

    # Center coordinates
    centered_coords = coords - center

    # Apply rotation
    rotated_coords = np.dot(centered_coords, rotation_matrix.T)

    # Move back to original position
    return rotated_coords + center

# Now you have a list of files in input_files
for pdb_file in input_files:
    outputFiles.append(pdb_file)
    for i in range(num_iterations):
        # Create a PDB parser
        parser = PDBParser()

        # Load the structure
        structure = parser.get_structure("target", pdb_file)

        chains_to_select = []
        for chain in structure[0]:
            for residue in chain:
                if residue.get_resname() == "HEM":
                    chains_to_select.append(chain.id)
                    break

        perturbed_structure = structure.copy()

        # Generate random displacements for x, y, and z coordinates
        dx = random.uniform(-max_displacement, max_displacement)
        dy = random.uniform(-max_displacement, max_displacement)
        dz = random.uniform(-max_displacement, max_displacement)

        # Generate a single random rotation angle for all hemes
        rotation_angle = random.uniform(0, max_rotation)

        # Iterate through the selected chains
        for chain_id in chains_to_select:
            chain = perturbed_structure[0][chain_id]

            # Process each HEM residue
            for residue in chain:
                if residue.get_resname() == "HEM":
                    # Get heme center (Fe) and normal vector
                    center, normal = get_heme_center_and_normal(residue)

                    # Apply rotation to all atoms in the HEM residue
                    for atom in residue:
                        # First rotate around normal vector using the same rotation_angle
                        new_coords = rotate_around_axis(atom.coord, center, normal, rotation_angle)

                        # Then apply translation
                        new_coords = new_coords + np.array([dx, dy, dz])

                        atom.set_coord(new_coords)

        # Save the modified structure to a new PDB file
        io = PDBIO()
        io.set_structure(perturbed_structure)
        outputFiles.append(os.path.join(output_dir, f"p2erturbed_{i+1}.pdb"))
        output_filename = os.path.join(output_dir, f"p2erturbed_{i+1}.pdb")
        io.save(output_filename)


random.shuffle(outputFiles)

In [ ]:
for pdb_file in input_files:
    outputFiles.insert(0, pdb_file) # Use insert(0, item) to prepend




In [ ]:
# prompt: use py3dmol to show the created structure with the hemes in stick format

import py3Dmol

def view_pdb(pdb_file):
  view = py3Dmol.view(width=800, height=600)
  view.addModel(open(pdb_file,'r').read(),'pdb')
  view.setStyle({'cartoon':{'color':'spectrum'}})
  view.setStyle({'resn':'HEM'},{'stick':{'colorscheme':'greenCarbon'}})
  view.zoomTo()
  return view

# Example usage with the first perturbed file (adjust the index as needed)
i=1
print(outputFiles[i])
view = view_pdb(outputFiles[i])
view.show()

./inputs/p2erturbed_21.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
import os
import numpy as np
from IPython.display import clear_output
import json
# Create the output directory if it doesn't exist
os.makedirs("./outputs/default", exist_ok=True)
#pdb_file = "./inputs/target-complex.pdb"
#outputFiles = [pdb_file]
#with open("./outputs/default/seqs/target-long.fa", "r") as infile, open("batch.fa", "a") as outfile:
#        outfile.write(infile.read())
cc=0
for file in outputFiles:
  print(f"Loaded {file}")
  for seed in range(10):
      temp = np.random.randint(100)/100.0 * .1
      print(f"Running seed {seed+1} with temp {temp}")
      sseed=np.random.randint(100000)+seed

      change =4*(10-seed)/10.0# np.random.randint(40)/10.0
      bias = f"A:{change/1.5:.2f},T:{change:.2f},R:{change:.2f},E:{change:.2f},Y:{change/2:.2f},G:{-1*change:.2f},S:{-1*change:.2f},K:{-1*change:.2f},D:{-1*change:.2f}"

      !python run.py --verbose 0 --model_type "ligand_mpnn" --checkpoint_ligand_mpnn "./model_params/ligandmpnn_v_32_030_25.pt" \
            --temperature {temp} --seed {sseed+1} --pdb_path {file} --out_folder "./outputs/default" --batch_size 10 --number_of_batches 5 \
            --bias_AA {bias}
      #clear_output()
      # Concatenate chloro.fa content to batch.fa
      outputFile = file.replace(".pdb", ".fa").replace('./inputs/', './outputs/default/seqs/')
      print(f"outputFile: {outputFile}")
      with open(outputFile, "r") as infile, open("/content/drive/MyDrive/batch.fa", "a") as outfile:
          outfile.write(infile.read())
      print (f"Finished seed {cc} : {seed+1} with temp {temp}")
      cc+=1



Loaded ./inputs/working2_Combin4.pdb
Running seed 1 with temp 0.07800000000000001
@> ProDy is configured: verbosity='none'
outputFile: ./outputs/default/seqs/working2_Combin4.fa
Finished seed 0 : 1 with temp 0.07800000000000001
Running seed 2 with temp 0.032
outputFile: ./outputs/default/seqs/working2_Combin4.fa
Finished seed 1 : 2 with temp 0.032
Running seed 3 with temp 0.005000000000000001
outputFile: ./outputs/default/seqs/working2_Combin4.fa
Finished seed 2 : 3 with temp 0.005000000000000001
Running seed 4 with temp 0.007000000000000001
outputFile: ./outputs/default/seqs/working2_Combin4.fa
Finished seed 3 : 4 with temp 0.007000000000000001
Running seed 5 with temp 0.005000000000000001
outputFile: ./outputs/default/seqs/working2_Combin4.fa
Finished seed 4 : 5 with temp 0.005000000000000001
Running seed 6 with temp 0.021
outputFile: ./outputs/default/seqs/working2_Combin4.fa
Finished seed 5 : 6 with temp 0.021
Running seed 7 with temp 0.011000000000000001
outputFile: ./outputs/defa

In [ ]:
# Define the maximum displacement (in Angstroms)
max_displacement = 2.0

# Define the number of iterations
num_iterations = 1

# Assuming ./inputs is the directory containing the files
input_files = glob.glob("./inputs/perturbed*.pdb")  # Adjust the pattern if needed

outputFiles = []
output_dir = "./inputs"
# Now you have a list of files in input_files
for pdb_file in input_files:
  outputFiles.append(pdb_file)
  continue
  for i in range(num_iterations):
    # Create a PDB parser
    parser = PDBParser()

    # Load the structure
    structure = parser.get_structure("target", pdb_file)

    chains_to_select = []
    for chain in structure[0]:
        for residue in chain:
            if residue.get_resname() == "HEM":
                chains_to_select.append(chain.id)
                break
    perturbed_structure = structure.copy()

    # Generate random displacements for x, y, and z coordinates
    dx = random.uniform(-max_displacement, max_displacement)
    dy = random.uniform(-max_displacement, max_displacement)
    dz = random.uniform(-max_displacement, max_displacement)

    # Iterate through the selected chains
    for chain_id in chains_to_select:
        chain = perturbed_structure[0][chain_id]

        # Iterate through all atoms in the selected chains
        for atom in chain.get_atoms():
            # Apply the displacement to the atom's coordinates
            atom.set_coord(atom.get_coord() + np.array([dx, dy, dz]))


    # Save the modified structure to a new PDB file
    io = PDBIO()
    io.set_structure(perturbed_structure)
    outputFiles.append(os.path.join(output_dir, f"p2erturbed_{i+1}.pdb"))
    output_filename = os.path.join(output_dir, f"p2erturbed_{i+1}.pdb")
    io.save(output_filename)


random.shuffle(outputFiles)


In [ ]:

def process_fasta_file(filename):
    """
    Loads the output file, skips the first two lines, and processes the remaining lines
    in FASTA-like format to create a dictionary.

    Args:
        filename: The path to the output file.

    Returns:
        A dictionary where keys are parameter strings and values are sequences.
        Returns None if the file is not found or an error occurs.
    """
    with open(filename, 'r') as f:
        lines = f.readlines()
        # Skip the first two lines
        lines = lines[2:]

        fasta_dict = {}
        i = 0
        cc=0
        while i < len(lines):
            params_line = lines[i].strip()
            if not params_line.startswith('>'):
                #Handle cases with missing or malformed headers
                i += 1
                continue
            try:
              sequence_line = lines[i + 1].strip()
            except:
              #Handle cases where no sequence is present
              i +=1
              continue

            params = params_line[1:].split(',')  # Remove the '>'
            sourceFile = params[0]
            params = {param.split('=')[0]: param.split('=')[1] for param in params[1:]}
            params['sourceFile'] = sourceFile
            fasta_dict[cc] ={ 'params':params, 'seq': sequence_line}
            i += 2
            cc+=1

        return fasta_dict


class HelixLinker:
    def __init__(self, model, excluded_chains=None):
        """
        Initialize linker for connecting helix chains.

        Args:
            model: BioPython Model object containing the helices
            excluded_chains: List of chain IDs to exclude (e.g., hemes)
        """
        self.model = model
        self.excluded_chains = excluded_chains or []
        self.chain_ends = {}  # Store terminal residues for each chain
        self.connections = []  # Store pairs of chain ends to connect

        # Standard amino acid geometry
        self.ca_c_length = 1.52    # Å
        self.c_n_length = 1.33     # Å
        self.n_ca_length = 1.46    # Å
        self.ca_ca_distance = 3.8  # Å
        self.ca_c_n_angle = 117.2  # degrees
        self.c_n_ca_angle = 121.7  # degrees
        self.peptide_omega = 180.0 # degrees

        # Initialize structure
        self._identify_chain_ends()
        self._find_closest_connections()

    def _identify_chain_ends(self):
        """Identify N and C terminal residues for each chain."""
        for chain in self.model:
            if chain.id in self.excluded_chains:
                continue

            residues = list(chain)
            if not residues:
                continue

            self.chain_ends[chain.id] = {
                'N': residues[0],
                'C': residues[-1]
            }

    def _find_closest_connections(self):
        """Find closest chain ends to connect."""
        chain_ids = list(self.chain_ends.keys())

        for i, chain1 in enumerate(chain_ids):
            distances = []
            for chain2 in chain_ids[i+1:]:  # Only look at chains we haven't considered yet
                end_carbon = self.chain_ends[chain1]['C']['C'].get_coord()
                start_n = self.chain_ends[chain2]['N']['N'].get_coord()

                distances.append({
                    'c2': chain2,
                    'distance': np.linalg.norm(end_carbon - start_n)
                })

            if distances:  # Only add if we found valid connections
                best_connection = min(distances, key=lambda x: x['distance'])
                self.connections.append({
                    'chain1': chain1,
                    'chain2': best_connection['c2'],
                    'distance': best_connection['distance']
                })

    def _calculate_residue_direction(self, residue):
        """Calculate the direction vector of a residue's backbone."""
        ca_pos = residue['CA'].get_coord()
        if 'C' in residue:
            c_pos = residue['C'].get_coord()
            direction = c_pos - ca_pos
        else:
            n_pos = residue['N'].get_coord()
            direction = ca_pos - n_pos
        return direction / np.linalg.norm(direction)

    def _evaluate_bezier(self, control_points, t):
        """Evaluate a cubic Bezier curve at parameter t."""
        p0, p1, p2, p3 = control_points
        return (
            (1-t)**3 * p0 +
            3*(1-t)**2 * t * p1 +
            3*(1-t) * t**2 * p2 +
            t**3 * p3
        )

    def _calculate_path_length(self, control_points, num_points=100):
        """Calculate the total length of the Bezier curve."""
        t = np.linspace(0, 1, num_points)
        points = np.array([self._evaluate_bezier(control_points, ti) for ti in t])

        # Calculate total length as sum of segments
        segments = np.diff(points, axis=0)
        lengths = np.sqrt(np.sum(segments**2, axis=1))
        return np.sum(lengths)

    def _optimize_control_points(self, start_pos, end_pos, start_dir, end_dir, num_residues):
        """Optimize control points to achieve desired path length."""
        target_length = (num_residues - 1) * self.ca_ca_distance
        distance = np.linalg.norm(end_pos - start_pos)

        def objective(x):
            # x contains scaling factors for control points
            scale1, scale2 = x
            p0 = start_pos
            p1 = start_pos + start_dir * (distance * scale1)
            p2 = end_pos - end_dir * (distance * scale2)
            p3 = end_pos

            path_length = self._calculate_path_length((p0, p1, p2, p3))
            return (path_length - target_length)**2

        # Optimize scaling factors
        result = minimize(objective, [0.33, 0.33], bounds=[(0.1, 0.9), (0.1, 0.9)])
        scale1, scale2 = result.x
        scale1, scale2 = (.33,.33)
        # Return optimized control points
        return (
            start_pos,
            start_pos + start_dir * (distance * scale1),
            end_pos - end_dir * (distance * scale2),
            end_pos
        )

    def generate_linker(self, connection ):
        """Generate backbone coordinates for a linker using an optimized Bezier curve."""
        chain1 = self.chain_ends[connection['chain1']]
        chain2 = self.chain_ends[connection['chain2']]

        start_res = chain1['C']
        end_res = chain2['N']
        start_direction =self._calculate_residue_direction(start_res)* (.5 + np.random.rand() * 2)
        end_direction = self._calculate_residue_direction(end_res) * (.5 + np.random.rand() * 2)

        start_ca = start_res['CA'].get_coord()
        end_ca = end_res['CA'].get_coord()

        distance = np.linalg.norm(end_ca - start_ca)
        # Generate a test path for the Bezier curve
        test_control_points = (start_ca, start_ca + start_direction * (distance / 3), end_ca - end_direction * (distance / 3), end_ca)
        test_path_length = self._calculate_path_length(test_control_points)
        num_residues = int(np.ceil(test_path_length / self.ca_ca_distance))



        # Optimize control points for target path length
        control_points = self._optimize_control_points(
            start_ca, end_ca, start_direction, end_direction, num_residues
        )

        # Generate points along optimized Bezier curve
        t = np.linspace(0, 1, num_residues)
        path = np.array([self._evaluate_bezier(control_points, ti) for ti in t])

        # Generate backbone atoms
        backbone = []
        for i in range(num_residues):
            if i == 0:
                prev_c = start_res['C'].get_coord()
                prev_ca = start_ca
            else:
                prev_c = backbone[i-1]['C']
                prev_ca = backbone[i-1]['CA']

            # Calculate tangent for better backbone placement
            if i < len(path) - 1:
                tangent = path[i+1] - path[i]
            else:
                tangent = end_direction
            tangent = tangent / np.linalg.norm(tangent)

            res_atoms = self._generate_backbone_atoms_with_direction(
                prev_c, prev_ca, path[i], tangent
            )
            backbone.append(res_atoms)

        return backbone

    def _generate_backbone_atoms_with_direction(self, prev_c, prev_ca, current_ca, direction):
        """Generate backbone atoms with proper geometry."""
        peptide_plane_normal = np.cross(prev_c - prev_ca, direction)
        peptide_plane_normal = peptide_plane_normal / np.linalg.norm(peptide_plane_normal)

        n_direction = np.cross(direction, peptide_plane_normal)
        n_pos = prev_c + self.c_n_length * n_direction

        ca_pos = current_ca

        c_direction = np.cross(direction, peptide_plane_normal)
        c_pos = ca_pos + self.ca_c_length * c_direction

        return {
            'N': n_pos,
            'CA': ca_pos,
            'C': c_pos
        }

    def connect_chains(self):
        """Connect all identified chain pairs with linkers."""
        connection=  min(self.connections, key=lambda x: x['distance'])
        if connection['distance'] > 25:
            #print('too far',connection['distance'])
            return False
        backbone_atoms = self.generate_linker(connection)

        # Get the chains to connect
        chain1_id = connection['chain1']
        chain2_id = connection['chain2']

        # Add linker residues to chain1
        last_res_id = max(int(res.id[1]) for res in self.model[chain1_id])

        # Add linker residues
        for i, atoms in enumerate(backbone_atoms):
            new_res_id = last_res_id + i + 1
            new_res = Residue.Residue((' ', new_res_id, ' '), 'GLY', '')

            for atom_name, coord in atoms.items():
                new_atom = Atom.Atom(
                    name=atom_name,
                    coord=coord,
                    bfactor=20.0,
                    occupancy=1.0,
                    altloc=' ',
                    fullname=atom_name,
                    serial_number=None,
                    element=atom_name[0]
                )
                new_res.add(new_atom)

            self.model[chain1_id].add(new_res)

        # Move residues from chain2 to chain1
        residues_to_move = list(self.model[chain2_id].get_residues())
        chain =self.model[chain1_id]
        new_res_id=max([max(int(res.id[1]) for res in chain), max(int(res.id[1]) for res in residues_to_move)])
        for res in residues_to_move:
            try:
                new_res_id +=1
                res.id = (' ', new_res_id, ' ')
                chain.add(res)
            except Exception as e:
                print(e)
                print("Error adding residue")

        # Remove chain2
        self.model.detach_child(chain2_id)
        #print(chain1_id,chain2_id)
        return True

def connectStructure(filename):
  parser = PDBParser(QUIET=True)
  structure = parser.get_structure('protein', filename)
  model = structure[0]

  # Save structure
  io = PDBIO()
  io.set_structure(model)
  io.save("connected_structure.pdb")


  shortest=0
  while shortest <30:

      parser = PDBParser(QUIET=True)
      structure = parser.get_structure('protein', 'connected_structure.pdb')
      model = structure[0]
      # Find all chains in model that have a HEM marker
      heme_chains = []
      for chain in model:
          for residue in chain:
              if residue.get_resname() == "HEM":
                  heme_chains.append(chain.id)
                  break

      linker = HelixLinker(model, excluded_chains=heme_chains)
      shortest = min(linker.connections, key=lambda x: x['distance'])['distance']
      minFound = linker.connect_chains( )
      if minFound==False:
          break

      # Save structure
      io = PDBIO()
      io.set_structure(model)
      io.save("connected_structure.pdb")


  return shortest


def pdb_to_fasta(pdb_file):
    """
    Loads a PDB file, extracts sequences from chains, and returns a FASTA string.

    Args:
        pdb_file: Path to the PDB file.

    Returns:
        A string in FASTA format, with each chain's sequence.
        Returns an empty string if there's an error.
    """

    parser = PDBParser()
    structure = parser.get_structure("protein", pdb_file)
    fasta_sequences = []

    for model in structure:
        for chain in model:
            sequence = ""
            for residue in chain:
                # Consider only standard amino acids (you might adjust this)
                if residue.get_resname() in ["ALA", "ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "GLY", "HIS", "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL"]:
                    sequence += residue.get_resname()[0]

            if sequence:
                record = SeqRecord(Seq(sequence), id=f"{pdb_file}_{chain.id}", description="")
                fasta_sequences.append(record)

    if fasta_sequences:
      fasta = []
      for seq in fasta_sequences:
        fasta .append( f'{seq.seq}')
      return (':'.join(fasta))
    else:
        return ""  # No valid sequences found




In [ ]:
import os
import numpy as np
from IPython.display import clear_output
import json
# Create the output directory if it doesn't exist
os.makedirs("./outputs/default", exist_ok=True)
#pdb_file = "./inputs/target-complex.pdb"
#outputFiles = [pdb_file]
#with open("./outputs/default/seqs/target-long.fa", "r") as infile, open("batch.fa", "a") as outfile:
#        outfile.write(infile.read())
cc=0
for seed in range(100):
    #add a random temp
    temp = np.random.randint(100)/100.0 * .1
    for file in outputFiles:
      print(f"Running seed {seed+1} with temp {temp}")
      sseed=np.random.randint(100000)+seed
      !python run.py --verbose 0 --model_type "ligand_mpnn" --checkpoint_ligand_mpnn "./model_params/ligandmpnn_v_32_030_25.pt" --temperature {temp} --seed {sseed+1} --pdb_path {file} --out_folder "./outputs/default" --batch_size 2 --number_of_batches 5
      #clear_output()
      # Concatenate chloro.fa content to batch.fa
      outputFile = file.replace(".pdb", ".fa").replace('./inputs/', './outputs/default/seqs/')
      print(f"outputFile: {outputFile}")
      with open(outputFile, "r") as infile, open("/content/drive/MyDrive/batch.fa", "a") as outfile:
          outfile.write(infile.read())
      print (f"Finished seed {cc} : {seed+1} with temp {temp}")

      fasta_data = process_fasta_file(outputFile)
      pdbs = []

      for key, value in fasta_data.items():
        try:
          pdb = f'/content/LigandMPNN/outputs/default/backbones/{value["params"]["sourceFile"]}_{key+1}.pdb'
          connectStructure(pdb)
          fasta_string = pdb_to_fasta('/content/LigandMPNN/connected_structure.pdb')
          value ['fasta'] = fasta_string
          pdbs.append(value)
          os.remove('/content/LigandMPNN/connected_structure.pdb')
          os.remove(pdb)
        except:
          pass
      fasta_out = ""
      for pbs in pdbs:
          pbs['params']['seq']=  pbs['seq']
          ps = json.dumps(pbs['params'])
          fasta = pbs['fasta']
          fasta_out += f">{ps} \n{fasta}\n"
      print(fasta_out)
      with open("/content/drive/MyDrive/batch_struct.fa", "a") as outfile:
          outfile.write(fasta_out)
      cc+=1





Streaming output truncated to the last 5000 lines.
>{" id": "4", " T": "0.032", " seed": "27886", " overall_confidence": "0.4421", " ligand_confidence": "0.5462", " seq_rec": "0.1816", "sourceFile": "perturbed_edge_366", "seq": "GVEKVGEALEKANKEEKKKAEEEVERLKKKLEELG:HLGPVLVALVKIAKAVKKLIEALKEIKEEKEKVKG:GVEVVGKALKILNKAEKEKAKIKVENEKELAKILG:GVAEVGKALEYKNKAFEKELKAKVEIEKEKAAVKG:GVEVVGKALEKKNKFKKEKAEAKAEIKKELAKIKG:GVEEVGKALKAKNKAKEAEGKIKVEVEKEKAAVKG:GVEVVGKALEALNKYKKAKAEAEVEAKKEEAKVKG:GVEVVGKALKAKNKAAEAKAKAEVELEKEKAAVKG:GVEVVGKWLKAKNKAEKKKAKAEVEVKKEEAKVKG:GVDVVGKALKGKNAAAEAKGKAELEAEKEKAAVKG:GVEVVGKWLEAENKAFEEKEKAEVEIEKEEAKVKG:GVDVVGKALKGLNAAAKAKAEAAVEAAKEKAAVAG:GKGVKKVYEACEKKAKEKKEKAKAEIEAEEAKVKG:GVDVVAKALKAAAAAAKAAAAAEVAAAAAAAAALG"} 
GVGLVGGALGLAALGGLLLAGGGVGALLLLLGGLG:HLGPVLVALVLIALAVLLLIGALLGILGGLGLVLG:GVGVVGLALLILALAGLGLALILVGAGLGLALILG:GVAGVGLALGTLALAPGLGLLALVGIGLGLAAVLG:GVGVVGLALGLLALPLLGLAGALAGILLGLALILG:GVGGVGLALLALALALGAGGLILVGVGLGLAAVLG:GVGVVGLALGALALTLLALAGAGVGALLGGALVLG:GVGVVGLALLAL